## Setup Data Fetching

In [1]:
import ta

import pandas as pd
import tensortrade.env.default as default

from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.instruments import USD, BTC, ETH, LTC
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order

## Fetch Historical Data

In [10]:
cdd = CryptoDataDownload()

coinbase_data = pd.concat([
    cdd.fetch("Coinbase", "USD", "BTC", "1h").add_prefix("BTC:"),
    cdd.fetch("Coinbase", "USD", "ETH", "1h").add_prefix("ETH:")
], axis=1)

bitstamp_data = pd.concat([
    cdd.fetch("Bitstamp", "USD", "BTC", "1h").add_prefix("BTC:"),
    cdd.fetch("Bitstamp", "USD", "LTC", "1h").add_prefix("LTC:")
], axis=1)

In [11]:
coinbase_data.head()

BTC:date  BTC:open  BTC:high  ...  ETH:low  ETH:close  ETH:volume
0 2017-07-01 11:00:00   2505.56   2513.38  ...    272.1     275.01   679358.87
1 2017-07-01 12:00:00   2509.17   2512.87  ...    271.0     274.83   824362.87
2 2017-07-01 13:00:00   2488.43   2488.43  ...    265.0     268.79  3010787.99
3 2017-07-01 14:00:00   2454.43   2473.93  ...    265.0     265.74  1702536.85
4 2017-07-01 15:00:00   2459.35   2475.00  ...    265.0     272.57  1500282.55

[5 rows x 12 columns]

In [12]:
bitstamp_data.head()

BTC:date  BTC:open  BTC:high  ...  LTC:low  LTC:close  LTC:volume
0 2017-07-01 11:00:00   2506.50   2510.62  ...    39.32      39.45     1957.48
1 2017-07-01 12:00:00   2500.00   2503.69  ...    39.18      39.57     2507.48
2 2017-07-01 13:00:00   2495.62   2495.62  ...    38.80      38.90    22019.12
3 2017-07-01 14:00:00   2449.01   2480.60  ...    38.77      39.06    30732.98
4 2017-07-01 15:00:00   2457.90   2471.88  ...    38.73      39.15    26053.18

[5 rows x 12 columns]

## Define Exchanges

An exchange needs a name, an execution service, and streams of price data in order to function properly.

The setups supported right now are the simulated execution service using simulated or stochastic data. More execution services will be made available in the future, as well as price streams so that live data and execution can be supported.

In [16]:


coinbase = Exchange("coinbase", service=execute_order)(
    Stream.source(list(coinbase_data['BTC:close']), dtype="float").rename("USD-BTC"),
    Stream.source(list(coinbase_data['ETH:close']), dtype="float").rename("USD-ETH")
)

bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream.source(list(bitstamp_data['BTC:close']), dtype="float").rename("USD-BTC"),
    Stream.source(list(bitstamp_data['LTC:close']), dtype="float").rename("USD-LTC")
)

Now that the exchanges have been defined we can define our features that we would like to include, excluding the prices we have provided for the exchanges.

## Define External Data Feed

Here we will define the feed to use whatever data you would like. From financial indicators to alternative datasets, they will all have to be defined and incorporated into the `DataFeed` provided to the environment.

In [ ]:
# Add all features for bitstamp BTC & ETH
coinbase_btc = coinbase_data.loc[:, [name.startswith("BTC") for name in coinbase_data.columns]]
coinbase_eth = coinbase_data.loc[:, [name.startswith("ETH") for name in coinbase_data.columns]]

ta.add_all_ta_features(
    coinbase_btc,
    colprefix="BTC:",
    **{k: "BTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)


with NameSpace("coinbase"):
    coinbase_streams = [
        Stream.source(list(coinbase_btc[c]), dtype="float").rename(c) for c in coinbase_btc.columns
    ]
    coinbase_streams += [
        Stream.source(list(coinbase_eth[c]), dtype="float").rename(c) for c in coinbase_eth.columns
    ]
    

# Add all features for bitstamp BTC & LTC
bitstamp_btc = bitstamp_data.loc[:, [name.startswith("BTC") for name in bitstamp_data.columns]]  
bitstamp_ltc = bitstamp_data.loc[:, [name.startswith("LTC") for name in bitstamp_data.columns]]

ta.add_all_ta_features(
    bitstamp_ltc,
    colprefix="LTC:",
    **{k: "LTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)

with NameSpace("bitstamp"):
    bitstamp_streams = [
        Stream.source(list(bitstamp_btc[c]), dtype="float").rename(c) for c in bitstamp_btc.columns
    ]
    bitstamp_streams += [
        Stream.source(list(bitstamp_ltc[c]), dtype="float").rename(c) for c in bitstamp_ltc.columns
    ]


feed = DataFeed(coinbase_streams + bitstamp_streams)

In [18]:
feed.next()

{'coinbase:/BTC:date': Timestamp('2017-07-01 11:00:00'),
 'coinbase:/BTC:open': 2505.56,
 'coinbase:/BTC:high': 2513.38,
 'coinbase:/BTC:low': 2495.12,
 'coinbase:/BTC:close': 2509.17,
 'coinbase:/BTC:volume': 287000.32,
 'coinbase:/BTC:volume_adi': 462887.3781183644,
 'coinbase:/BTC:volume_obv': nan,
 'coinbase:/BTC:volume_cmf': 0.5388828039430464,
 'coinbase:/BTC:volume_fi': nan,
 'coinbase:/BTC:volume_em': nan,
 'coinbase:/BTC:volume_vpt': -190920.02711825827,
 'coinbase:/BTC:volume_nvi': 1000.0,
 'coinbase:/BTC:volatility_atr': 85.51648155760596,
 'coinbase:/BTC:volatility_bbh': 2509.17,
 'coinbase:/BTC:volatility_bbl': 2509.17,
 'coinbase:/BTC:volatility_bbm': 2509.17,
 'coinbase:/BTC:volatility_bbhi': 0.0,
 'coinbase:/BTC:volatility_bbli': 0.0,
 'coinbase:/BTC:volatility_kcc': 2505.89,
 'coinbase:/BTC:volatility_kch': 2524.15,
 'coinbase:/BTC:volatility_kcl': 2487.6299999999997,
 'coinbase:/BTC:volatility_kchi': 0.0,
 'coinbase:/BTC:volatility_kcli': 0.0,
 'coinbase:/BTC:volatili

## Portfolio

Make the portfolio using the any combinations of exchanges and intruments that the exchange supports

In [19]:
portfolio = Portfolio(USD, [
    Wallet(coinbase, 10000 * USD),
    Wallet(coinbase, 10 * BTC),
    Wallet(coinbase, 5 * ETH),
    Wallet(bitstamp, 1000 * USD),
    Wallet(bitstamp, 5 * BTC),
    Wallet(bitstamp, 3 * LTC),
])

## Environment

In [20]:
env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="simple",
    feed=feed,
    window_size=15,
    enable_logger=False
)

In [22]:
env.observer.feed.next()

{'internal': {'coinbase:/USD-BTC': 2509.17,
  'coinbase:/USD-ETH': 275.01,
  'coinbase:/USD:/free': 10000.0,
  'coinbase:/USD:/locked': 0.0,
  'coinbase:/USD:/total': 10000.0,
  'coinbase:/BTC:/free': 10.0,
  'coinbase:/BTC:/locked': 0.0,
  'coinbase:/BTC:/total': 10.0,
  'coinbase:/BTC:/worth': 25091.7,
  'coinbase:/ETH:/free': 5.0,
  'coinbase:/ETH:/locked': 0.0,
  'coinbase:/ETH:/total': 5.0,
  'coinbase:/ETH:/worth': 1375.05,
  'bitstamp:/USD-BTC': 2500.0,
  'bitstamp:/USD-LTC': 39.45,
  'bitstamp:/USD:/free': 1000.0,
  'bitstamp:/USD:/locked': 0.0,
  'bitstamp:/USD:/total': 1000.0,
  'bitstamp:/BTC:/free': 5.0,
  'bitstamp:/BTC:/locked': 0.0,
  'bitstamp:/BTC:/total': 5.0,
  'bitstamp:/BTC:/worth': 12500.0,
  'bitstamp:/LTC:/free': 3.0,
  'bitstamp:/LTC:/locked': 0.0,
  'bitstamp:/LTC:/total': 3.0,
  'bitstamp:/LTC:/worth': 118.35000000000001,
  'net_worth': 50085.1},
 'external': {'coinbase:/BTC:date': Timestamp('2017-07-01 11:00:00'),
  'coinbase:/BTC:open': 2505.56,
  'coinbase